In [22]:
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
from skimage import data, filters, draw
import matplotlib.pyplot as plt
from skimage.draw import rectangle
from skimage.morphology import disk
from skimage import morphology
plt.rcParams['text.color'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
import math
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['font.size'] = 18
import plotly.graph_objects as go
from plotly.subplots import make_subplots
output_folder_csv = "../output/csv"
video_name = "010_001_test"
from cv_helper import CvHelper
df_raw_angles = pd.read_csv(os.path.join(output_folder_csv, f"{video_name}.csv"))
df_raw_angles

,frame,angle_0,angle_1,angle_2,i_p_0_0_x,i_p_0_0_y,i_p_0_x,i_p_0_y,i_p_1_x,i_p_1_y,i_p_2_x,i_p_2_y,i_p_3_x,i_p_3_y,time
0,0,152.760279,145.150848,161.911018,328,645,431,502,589,375,673,374,762,409,0.000000
1,1,152.760279,145.150848,163.176450,328,645,431,502,589,375,673,374,762,408,0.002083
2,2,152.760279,145.150848,163.176450,328,645,431,502,589,375,673,374,762,408,0.004167
3,3,152.760279,145.409518,162.917780,328,645,431,502,589,375,673,374,762,408,0.006250
4,4,152.760279,145.409518,162.917780,328,645,431,502,589,375,673,374,762,408,0.008333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10313,10313,137.460719,75.356557,175.971736,332,662,405,501,554,366,585,428,588,522,21.485417
10314,10314,136.132541,74.892455,178.058514,332,662,406,501,556,367,585,429,588,523,21.487500
10315,10315,136.132541,74.892455,176.522213,332,662,406,501,556,367,585,429,586,524,21.489583
10316,10316,136.486214,72.434985,177.160786,332,662,406,501,556,367,584,430,583,525,21.491667


In [23]:
from cv_helper import CvHelper

In [24]:
row = df_raw_angles[0:1]

# Create new point along on horizontal line with first point
row["marker_a_x"] = row["i_p_0_0_x"] + 100
row["marker_a_y"]  = row["i_p_0_0_y"]

/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_2401/2830361.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["marker_a_x"] = row["i_p_0_0_x"] + 100
/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_2401/2830361.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["marker_a_y"]  = row["i_p_0_0_y"]


In [25]:
# Calculate angle
shift_angle =  CvHelper.calculate_angle(
    (
        (int(row["i_p_0_x"]), int(row["i_p_0_y"])), (int(row["i_p_0_0_x"]), int(row["i_p_0_0_y"]))
    ),
    (
        (int(row["i_p_0_0_x"]), int(row["i_p_0_0_y"])), (int(row["marker_a_x"]), int(row["marker_a_y"]))
    )
)

-10300 -14300
54.235559093165925
-1


In [26]:
# Rotate points points
rotated_points = CvHelper.rotate_coordinates([(int(row["i_p_0_x"]), int(row["i_p_0_y"])),
                                     (int(row["i_p_1_x"]), int(row["i_p_1_y"])),
                                     (int(row["i_p_2_x"]), int(row["i_p_2_y"])),
                                     (int(row["i_p_3_x"]), int(row["i_p_3_y"]))
                                     ], -54, (int(row["i_p_0_0_x"]),int(row["i_p_0_0_y"])))

In [27]:
rotated_points

array([[504.23131118, 644.27545934],
       [699.84653933, 697.45141741],
       [750.02951752, 764.82105969],
       [774.02681017, 857.39605602]])

In [28]:
rotated_points

array([[504.23131118, 644.27545934],
       [699.84653933, 697.45141741],
       [750.02951752, 764.82105969],
       [774.02681017, 857.39605602]])

In [35]:
def rotate_frame_tracked_points(row):
    # Create new point along on horizontal line with first point
    row["marker_a_x"] = row["i_p_0_0_x"] + 100
    row["marker_a_y"]  = row["i_p_0_0_y"]

    # Calculate angle
    shift_angle =  CvHelper.calculate_angle(
        (
            (int(row["i_p_0_x"]), int(row["i_p_0_y"])), (int(row["i_p_0_0_x"]), int(row["i_p_0_0_y"]))
        ),
        (
            (int(row["i_p_0_0_x"]), int(row["i_p_0_0_y"])), (int(row["marker_a_x"]), int(row["marker_a_y"]))
        )
    )
    print(shift_angle)
    # Rotate points
    rotated_points = CvHelper.rotate_coordinates([(int(row["i_p_0_x"]), int(row["i_p_0_y"])),
                                                  (int(row["i_p_1_x"]), int(row["i_p_1_y"])),
                                                  (int(row["i_p_2_x"]), int(row["i_p_2_y"])),
                                                  (int(row["i_p_3_x"]), int(row["i_p_3_y"]))
                                                  ], (180 - shift_angle) * -1, (int(row["i_p_0_0_x"]),int(row["i_p_0_0_y"])))

    # Shifting point to central zero
    rotated_points[:,0] = rotated_points[:,0] - rotated_points[0,0]
    rotated_points[:,1] = rotated_points[:,1] - rotated_points[0,1]

    return rotated_points

In [36]:
rotate_frame_tracked_points(row)

-10300 -14300
54.235559093165925
-1
125.76444090683407


/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_2401/1051738513.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["marker_a_x"] = row["i_p_0_0_x"] + 100
/var/folders/mt/zbsgqls91d5c1dt725ckq4lc0000gn/T/ipykernel_2401/1051738513.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["marker_a_y"]  = row["i_p_0_0_y"]


array([[  0.        ,   0.        ],
       [195.39495421,  53.97973572],
       [245.30053349, 121.55512441],
       [268.91702251, 214.22799772]])